# Coursera Capstone Project
Anthony Suárez

This notebook is to work on my Capstone for the IBM Data Science Specialization.

## Week 1

In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
print("Hello Coursera Capstone Project!")

Hello Coursera Capstone Project!


## Week 3

### Collect data about Toronto neighborhoods

I will use Beautiful Soup to do web scraping and get data from Wikipedia.

In [3]:
page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_html = requests.get(page_url, timeout=10)

page_html

<Response [200]>

In [4]:
toronto_soup = BeautifulSoup(page_html.content)
# print(toronto_soup.prettify())

We are interested in the Toronto - 103 FSAs table, which has the classes "wikitable sortable"

In [5]:
fsas_table = toronto_soup.find("table", {"class": "wikitable sortable"})
# districts_table.__dict__

In [6]:
fsas_df = pd.read_html(str(fsas_table))
fsas_df = fsas_df[0]
fsas_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Next, we need to drop the rows with Borough not assigned

In [7]:
rows_not_assigned = fsas_df[fsas_df["Borough"] == "Not assigned"]
rows_not_assigned.index

Int64Index([  0,   1,   7,  10,  15,  16,  19,  24,  25,  28,  29,  33,  34,
             35,  37,  38,  42,  43,  44,  51,  52,  53,  60,  61,  62,  69,
             70,  71,  78,  79,  87,  88,  96,  97, 101, 105, 106, 110, 115,
            118, 119, 123, 124, 125, 127, 128, 131, 132, 133, 134, 136, 137,
            140, 141, 145, 146, 149, 150, 154, 155, 158, 159, 161, 162, 163,
            164, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 179],
           dtype='int64')

In [8]:
fsas_df = fsas_df.drop(rows_not_assigned.index, axis=0).reset_index().drop(columns=["index"])
fsas_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


If a neighborhood doesn't have a name assigned, we will assume it's the same name as its borough.

In [9]:
for row in fsas_df.iterrows():
    i = row[0]
    neighborhood = row[1]["Neighbourhood"]
    if neighborhood == "Not assigned":
        fsas_df.iloc[i]["Neighbourhood"] = fsas_df.iloc[i]["Borough"]
        
    break
        
fsas_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
fsas_df.rename(columns={"Neighbourhood": "Neighborhoods"}, inplace=True)
fsas_df.head()

,Postal Code,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now that we have parsed the table from Wikipedia, we have to get the location from each neighborhood.

In [11]:
import geocoder

In [12]:
def find_coords(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    attempts = 0
    while(lat_lng_coords is None or attempts < 100):
        attempts += 1
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    return lat_lng_coords[0], lat_lng_coords[1]

As the package used is unreliable, I used the csv file provided by Coursera

In [19]:
coords = pd.read_csv("data/Geospatial_Coordinates.csv")
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now, we need to combine theh coords and fsas dataframes

In [23]:
neighborhoods_data = pd.merge(fsas_df, coords, on="Postal Code")
neighborhoods_data.head()

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Visualize Toronto Neighborhoods

In [24]:
# !pip install folium
import folium

In [28]:
toronto_coords = [43.651070, -79.347015]
toronto_map = folium.Map(location=toronto_coords,
                         tiles='Stamen Toner',
                         zoom_start=10.5)

for i, row in neighborhoods_data.iterrows():
    marker = folium.CircleMarker(
        location=[row.Latitude, row.Longitude],
        popup=row.Neighborhoods,
        color='crimson',
        radius=5,
        fill=True
    ).add_to(toronto_map)

toronto_map

### Explore venues with Foursquare API

In [31]:
from dotenv import load_dotenv # Use pip to install python-dotenv package
import os

In [32]:
load_dotenv()

# If you are going to run this notebook, please add your own Foursquare credentials in a .env file.
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [35]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500

In [36]:
# function that extracts the category of the venue. By Coursera.
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

The next function returns nearby venues given a latitude and longitude

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

As the next piece of code takes some time to run, I exported the results to a .csv file.

```python
venues = getNearbyVenues(names=neighborhoods_data['Borough'],
                                      latitudes=neighborhoods_data['Latitude'],
                                      longitudes=neighborhoods_data['Longitude'])

venues.to_csv('data/venues.csv', index=False)
```

In [44]:
# Neighborhood refers to the borough
venues = pd.read_csv('data/venues.csv')
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [45]:
venues['Venue Category'].unique().shape

(273,)

Perfect! Now we have used Foursquare data to obtain information about venues close to each neighborhood in Toronto. The next step is processing the obtained data to feed it into our clustering algorithm.

### Data wrangling

It's time to prepare our data so it can be fed into the machine learning algorithm.

In [46]:
# From IMB/Coursera Course:
# one hot encoding
neighborhoods_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhoods_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
first_col = neighborhoods_onehot.pop("Neighborhood")
neighborhoods_onehot.insert(0, "Neighborhood", first_col)

In [47]:
neighborhoods_grouped = neighborhoods_onehot.groupby("Neighborhood").mean().reset_index()
neighborhoods_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009615,...,0.000000,0.0,0.000000,0.000000,0.009615,0.000000,0.000000,0.000000,0.0,0.009615
1,Downtown Toronto,0.0,0.000801,0.000801,0.000801,0.000801,0.001603,0.001603,0.000801,0.013622,...,0.002404,0.0,0.011218,0.001603,0.004006,0.000000,0.006410,0.000000,0.0,0.005609
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025210,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.008403,0.000000,0.0,0.016807
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.0,0.012658
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.0,0.000000


Print each neighborhoods most common venues

In [48]:
num_top_venues = 5

for hood in neighborhoods_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighborhoods_grouped[neighborhoods_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.07
2            Park  0.06
3            Café  0.06
4     Pizza Place  0.05


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.05
2          Restaurant  0.04
3               Hotel  0.03
4  Seafood Restaurant  0.02


----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1    Greek Restaurant  0.06
2  Italian Restaurant  0.04
3             Brewery  0.04
4              Bakery  0.03


----East York----
            venue  freq
0    Intersection  0.05
1     Coffee Shop  0.05
2            Bank  0.05
3  Sandwich Place  0.04
4            Park  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1     Coffee Shop  0.07
2  Sandwich Place  0.07
3        Pharmacy  0.05
4             Gym  0.04


----Mississauga----
                      venue  freq
0                     Hotel  0.15
1               Coffee Shop 

Put it into a dataframe

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhoods_grouped['Neighborhood']

for ind in np.arange(neighborhoods_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhoods_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Restaurant,Bagel Shop,Italian Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Brewery,Park,Ice Cream Shop,Restaurant,Bookstore,Café,Pub
3,East York,Intersection,Coffee Shop,Bank,Burger Joint,Sandwich Place,Sporting Goods Shop,Pizza Place,Park,Indian Restaurant,Pet Store
4,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Fast Food Restaurant,Grocery Store,Gym,Liquor Store,Café,Convenience Store


## Cluster Neighborhoods

I will use KMeans to cluster the neighborhoods. Also, it's important to note that I don't have the coords to every neighborhood, so I am using the coords of their borough.

In [63]:
from sklearn.cluster import KMeans

In [64]:
# set number of clusters
kclusters = 5

neighborhoods_grouped_clustering = neighborhoods_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 4, 3, 2, 0, 2, 1])

In [70]:
# add clustering labels

try:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except:
    pass

neighborhoods_merged = neighborhoods_data.rename(columns={"Borough": "Neighborhood"})

neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood').rename(columns={"Neighborhood":"Borough"})

neighborhoods_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Grocery Store,Park,Bank,Juice Bar
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Grocery Store,Park,Bank,Juice Bar
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Grocery Store,Park,Bank,Juice Bar
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant


In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors

The following map represents each neighborhood/borough with a marker in the map. Neighbourhoods with the same color are in the same cluster

In [74]:
# create map
map_clusters = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Borough'], neighborhoods_merged['Cluster Labels']):
    if np.isnan(cluster):
        cluster = 5
    else:
        cluster = int(cluster)

    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

End of part 3